In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

**keys**: homogeneous Neumann bvp, hypersingular operator $D$

Neumann Laplace Indirect Method
======================================================

Consider the Neumann boundary value problem 

$$ \left\{ \begin{array}{rlc l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_1 u&=& j, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

The solution $u\in H^1(\Omega)$ of the above bvp can be written in the following form (representation forumula) 

$$ u(x) =  \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{ \langle n(y), x-y \rangle }{\| x-y\|^3} } \, m(y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{DL}(m) }}\,.$$ 


**NGSolve Solution** 

Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(2)

Define the finite element space for $H^{\frac12}(\Gamma)$ for given geometry :  

In [ ]:
fesH1 = H1(mesh, order=2, definedon=mesh.Boundaries(".*"))
uH1,vH1 = fesH1.TnT()
print ("ndof H1 = ", fesH1.ndof)

**Boundary Integral Equation** 

Let's carefully apply the Neumann trace $\gamma_1$ to the representation formula and solve the resulting boundary integral equation for $m \in H^{\frac12}(\Gamma)$ by discretisation of the following variational formulation: 

$$\begin{array}{r c ccc} \forall \, v\in H^{\frac12}(\Gamma): && \displaystyle \int\limits_\Gamma v \cdot \gamma_1 \left(\mathrm{DL}(m) \right) \, \mathrm d s  &=&  \displaystyle \int\limits_\Gamma j \cdot  v \,\mathrm d s \\[2ex] & \Rightarrow & \mathrm{D}\,\mathrm{m} &=& \mathrm{M} \, \mathrm j \,. \end{array} $$ 


The discretisation of the variational formulation leads to a system of linear equations, ie 

$$ \left( \mathrm{D} + \mathrm{S}\right) \, \mathrm{m}= \mathrm{M}\, \mathrm j\,, $$ 

where $\mathrm{D}$ is the hypersingular operator and the stabilisation $(\mathrm D + \mathrm{S})$ is regular and symmetric and $\mathrm j$ is the given Neumann data.

In [ ]:
uexa = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )
graduexa = CF( (uexa.Diff(x), uexa.Diff(y), uexa.Diff(z)) )

n = specialcf.normal(3)
j = graduexa*n
rhs = LinearForm(j*vH1.Trace()*ds(bonus_intorder=3)).Assemble()

In [ ]:
vH1m1 = LinearForm(vH1*1*ds(bonus_intorder=3)).Assemble()
S = (BaseMatrix(Matrix(vH1m1.vec.Reshape(1))))@(BaseMatrix(Matrix(vH1m1.vec.Reshape(fesH1.ndof))))

The assembly of the hypersingular operator $D$ is declarative and close to their actual implementations (explicit formula are given below).

In [ ]:
D = LaplaceSL(curl(uH1)*ds) * curl(vH1)*ds

In [ ]:
m = GridFunction(fesH1)
pre = BilinearForm(uH1*vH1*ds).Assemble().mat.Inverse(freedofs=fesH1.FreeDofs()) 
with TaskManager(): 
    CG(mat = D.mat+S, pre=pre, rhs = rhs.vec, sol=m.vec, tol=1e-8, maxsteps=200, initialize=False, printrates=True)

In [ ]:
Draw (m, mesh, draw_vol=False, order=3);

**TODO Evaluation of the solution**

**Details** 

The hypersingular operator is implemented as follows (reference below)

$$ D_{ij} = \big\langle v_j, \gamma_1 \mathrm{DL}(u_i) \big\rangle = \frac{1}{4\pi} \int\limits_\Gamma\int\limits_\Gamma \frac{ \langle \mathrm{\boldsymbol {curl}}_\Gamma  \,u_i(x), \mathrm{\boldsymbol{curl}}_\Gamma\, v_j(y) \rangle}{\|x-y\|} \, \mathrm{d} s_{y} \, \mathrm{d} s_x $$ 

This explicit represenation of $D$ is the singular layer potential operator with vector valued surface curl operator applied on trial and test functions. You may assemble $D$ it by

```
    D = LaplaceSL(curl(u)*ds)*curl(v)*ds
```

**References**

For details on the explicit representation of the hypersingular operator have a look into [Numerische Näherungsverfahren für elliptische Randwertprobleme](https://link.springer.com/book/10.1007/978-3-322-80054-1), p.127, p.259 (1st edition).